# Lesson 2: Models, Prompts and Output Parsers

https://learn.deeplearning.ai/langchain/lesson/2/models,-prompts-and-parsers

- Direct API calls to OpenAI
- API calls through LangChain:
  - Prompts
  - Models
  - Output parsers


In [1]:
import os

import openai
from dotenv import find_dotenv, load_dotenv
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [2]:
load_dotenv(find_dotenv())

try:
    os.environ["OPENAI_API_KEY"]
except Exception:
    raise Exception("No OpenAI key found!")

In [6]:
CHAT_GPT_MODEL = "glm-4"
BASE_URL = "https://open.bigmodel.cn/api/paas/v4/"

Direct API calls to OpenAI


In [9]:
def get_completion(prompt: str, model=CHAT_GPT_MODEL) -> str | None:
    messages = [{"role": "user", "content": prompt}]

    response = openai.Client(base_url=BASE_URL).chat.completions.create(
        messages=messages,
        model=model,
        temperature=0.01,
    )

    return response.choices[0].message.content

In [10]:
get_completion("What is 1+1?")

'1+1 equals 2. This is a basic arithmetic operation representing the sum of two single units.'

In [11]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = "American English in a calm and respectful tone"

prompt = f"""Translate the text \
that is delimited by triple backticks \
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [12]:
get_completion(prompt)

"Oh no, I'm quite frustrated as my blender lid accidentally came off, causing my kitchen walls to be splattered with smoothie. To add to the inconvenience, the warranty does not cover the cleaning expenses for my kitchen. I could really use your assistance with this matter, my friend."

Do the same thing with LangChain

In [13]:
chat = ChatOpenAI(temperature=0.01, model=CHAT_GPT_MODEL, base_url=BASE_URL)

In [14]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [15]:
prompt_template = ChatPromptTemplate.from_template(template=template_string)

In [16]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [17]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [18]:
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [19]:
customer_messages = prompt_template.format_messages(
    style=customer_style,
    text=customer_email,
)

print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [20]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [22]:
# API call is made here!
customer_response = chat.invoke(customer_messages)

print(customer_response)

content="Oh no, I'm quite frustrated as my blender lid accidentally came off, causing my kitchen walls to be splattered with smoothie. To add to the inconvenience, the warranty doesn't cover the cleaning expenses. I could really use your assistance with this matter, my friend." response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 87, 'total_tokens': 144}, 'model_name': 'glm-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-dbd5cddc-ee9a-467a-bc70-1f55f25419a6-0' usage_metadata={'input_tokens': 87, 'output_tokens': 57, 'total_tokens': 144}


In [23]:
print(customer_response.content)

Oh no, I'm quite frustrated as my blender lid accidentally came off, causing my kitchen walls to be splattered with smoothie. To add to the inconvenience, the warranty doesn't cover the cleaning expenses. I could really use your assistance with this matter, my friend.


In [24]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [25]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply,
)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [26]:
# API call is made here!
service_response = chat.invoke(service_messages)

print(service_response.content)

Arr, me hearties! Here be the translation in a polite piratey tone:

```
Ahoy there, fine customer! T'would seem that the warranty doesn't extend to cover the cleansin' costs for yer galley, on account of the fact that the misuse of the blender – by forgoin' to secure the lid prior to its activation – befallen upon yer own doing. Aye, 'tis a sorry turn of events, that. Fare thee well, and may the winds be ever at yer back!
```


### Output Parsers

define how we would like the LLM output to look like


```python
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}
```


In [27]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [28]:
prompt_template = ChatPromptTemplate.from_template(template=review_template)

print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [29]:
messages = prompt_template.format_messages(text=customer_review)

chat = ChatOpenAI(temperature=0.01, model=CHAT_GPT_MODEL, base_url=BASE_URL)

response = chat.invoke(messages)

print(response.content)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [30]:
type(response.content)

str

Running
```python
response.content["gift"]
```
would fail because `response.content` is a str

In [31]:
gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.",
)

delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.",
)

price_value_schema = ResponseSchema(
    name="price_value",
    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.",
)

In [32]:
response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

In [33]:
output_parser = StructuredOutputParser.from_response_schemas(
    response_schemas=response_schemas
)

In [34]:
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [35]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

In [36]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(
    text=customer_review, format_instructions=format_instructions
)

In [37]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [38]:
response = chat.invoke(messages)

print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [39]:
output_dict = output_parser.parse(text=response.content)

In [40]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [41]:
output_dict["delivery_days"]

'2'